In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum ,avg, count
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .appName("Beer Reviews Preprocessing") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

24/11/11 17:12:22 WARN Utils: Your hostname, Abhays-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.36 instead (on interface en0)
24/11/11 17:12:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 17:12:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/11 17:12:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
reviews = spark.read.csv(path="downloaded_data/reviews.csv", header=True, schema="beer_id INT, username STRING, date DATE, text STRING, look DOUBLE, smell DOUBLE, taste DOUBLE, feel DOUBLE, overall DOUBLE, score DOUBLE")

In [4]:
grouped_reviews = reviews.repartition(4).drop("username", "date", "text") \
        .dropna(subset=["look", "smell", "taste", "feel", "overall", "score"]) \
        .groupby("beer_id").agg(
            count("*").alias("count"),
            avg("look").alias("look"),
            avg("smell").alias("smell"),
            avg("taste").alias("taste"),
            avg("feel").alias("feel"),
            avg("overall").alias("overall"),
            avg("score").alias("score")
        ).filter("count > 3")

In [5]:
beers = spark.read.csv(path="downloaded_data/beers.csv", header=True, inferSchema=True)

In [6]:
breweries = spark.read.csv(path="downloaded_data/breweries.csv", header=True, inferSchema=True)
breweries = breweries.drop("country", "notes", "types")

In [7]:
data = beers.join(grouped_reviews, beers.id == grouped_reviews.beer_id, "inner").drop("state", "notes", "beer_id")

In [8]:
final = data.join(breweries, data.brewery_id == breweries.id, "inner")

In [34]:
df = final.toPandas()

In [35]:
df = df.drop(columns=["city", "state"]).rename(columns={"name":"beer"}).dropna(subset="abv")

duplicate_columns = df.columns.duplicated()
df = df.loc[:, ~duplicate_columns]

In [36]:
df2 = df[df.duplicated(subset="beer")]
df2

,id,beer,brewery_id,country,style,availability,abv,retired,count,look,smell,taste,feel,overall,score,brewery
112,1238,Pale Ale,403,US,American Pale Ale (APA),Summer,5.8,f,47,3.654255,3.452128,3.494681,3.489362,3.638298,3.522766,"Pyramid Breweries, Inc."
195,2094,Pale Ale,423,US,American Pale Ale (APA),Year-round,5.4,f,556,3.737410,3.514388,3.652428,3.658273,3.795414,3.654388,Boulevard Brewing Co.
401,4798,Pale Ale,1683,US,American Pale Ale (APA),Year-round,5.4,f,187,3.667112,3.326203,3.457219,3.517380,3.584225,3.470588,Santa Fe Brewing Company
409,4866,Hefeweizen,1146,US,German Hefeweizen,Spring,4.8,f,17,3.735294,3.808824,3.882353,3.647059,3.779412,3.811765,Goose Island Beer Co.
429,5095,Pale Ale,778,US,American Pale Ale (APA),Year-round,4.2,t,7,3.571429,3.000000,3.285714,3.214286,3.357143,3.241429,Pioneer Brewing Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109303,365897,Dad Jokes,48592,US,American Imperial IPA,Rotating,8.3,f,15,4.183333,4.216667,4.266667,4.250000,4.216667,4.238667,Blackstack Brewing
109317,366944,What The Fluff?,49534,US,American Imperial Stout,Rotating,12,f,6,4.375000,4.333333,4.333333,4.291667,4.375000,4.341667,Barrel Theory Beer Company
109330,368401,Pale Ale,53867,US,American Pale Ale (APA),Rotating,5.8,f,4,4.125000,4.062500,4.125000,3.937500,4.125000,4.092500,Dynasty Brewing
109332,368720,Brut IPA,47157,US,American IPA,Rotating,6.5,f,5,4.000000,4.150000,4.100000,4.050000,4.050000,4.094000,New Heights Brewery


In [37]:
df2["beer"] = df2["brewery"] + " "+df2["beer"]
df3 = df.merge(df2[['id', 'beer']], on="id", how="left")

/var/folders/bh/xbydwd1s4h1d4fmjlbfcrbwc0000gn/T/ipykernel_48789/1274657400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["beer"] = df2["brewery"] + " "+df2["beer"]


In [38]:
df3['beer_x'] = df3.apply(lambda row: row['beer_y'] if pd.notnull(row['beer_y']) else row['beer_x'], axis=1)
df = df3.drop(columns="beer_y").rename(columns={"beer_x":"beer"})

In [41]:
df.to_csv("processed_data/merged_list.csv", index=False, header=True)